# P7 (Соціальне забезпечення та соціальний захист населення): 1/5

In [1]:
import pandas as pd
from src.utils import normalize_parameter, save_data
from src.utils import PATH_INTERIM

## p7_01
### кількість витрачених коштів на придбання технічних та інших засобів реабілітації у розрахунку на одну особу з інвалідністю

In [2]:
df_P07_002 = pd.read_excel(PATH_INTERIM / "P7" / "P07_002.xlsx")
df_P07_002["p7_01_raw"] = (
    df_P07_002["Кількість технічних засобів реабілітації"] / 
    df_P07_002["Кількість осіб"]
)
df_P07_002["p7_01"] = normalize_parameter(df_P07_002["p7_01_raw"])

feature_range=(0, 1); fill_na=True; array_bounds=(0.0, 0.024266365688487584); normalization_bounds=(0.0, 0.024266365688487584)


## p7_02
### частка дітей, що перебуває на обліку в інклюзивно-ресурсних центрах
## p7_06
### частка дітей, що здобуває освіту в інклюзивних групах ЗДО та інклюзивних класах ЗЗСО

In [3]:
relevant_columns = [
    "Кількість дітей на території обслуговування ІРЦ",
    "Кількість дітей, які перебувають на обліку в ІРЦ",
    "Кількість дітей у віці 2-5 років, які здобувають освіту в інклюзивних групах ЗДО",
    "Кількість дітей у віці 6-18 років, які здобувають освіту в інклюзивних класах ЗЗСО"
]

df_P07_004 = pd.read_excel(PATH_INTERIM / "P7" / "P07_004.xlsx")
df_P07_004_fin = df_P07_004.groupby("region", as_index=False)[relevant_columns].sum()
df_P07_004_fin["Здобувають освіту в інкл. класах ЗДО/ЗЗСО"] = (
    df_P07_004_fin["Кількість дітей у віці 2-5 років, які здобувають освіту в інклюзивних групах ЗДО"] +
    df_P07_004_fin["Кількість дітей у віці 6-18 років, які здобувають освіту в інклюзивних класах ЗЗСО"]
)

df_P07_004_fin["p7_02_raw"] = (
    df_P07_004_fin["Кількість дітей, які перебувають на обліку в ІРЦ"] / 
    df_P07_004_fin["Кількість дітей на території обслуговування ІРЦ"]
)

df_P07_004_fin["p7_06_raw"] = (
    df_P07_004_fin["Здобувають освіту в інкл. класах ЗДО/ЗЗСО"] / 
    df_P07_004_fin["Кількість дітей на території обслуговування ІРЦ"]
)

df_P07_004_fin["p7_02"] = normalize_parameter(df_P07_004_fin["p7_02_raw"], min_bound=0)
df_P07_004_fin["p7_06"] = normalize_parameter(df_P07_004_fin["p7_06_raw"], min_bound=0, max_bound=1)

feature_range=(0, 1); fill_na=True; array_bounds=(0.0047496335465501294, 0.032650321163205834); normalization_bounds=(0, 0.032650321163205834)
feature_range=(0, 1); fill_na=True; array_bounds=(0.001974260210472012, 0.009047495537280868); normalization_bounds=(0, 1)


## Група 1: p7_01, люди з інвалідністю
## Група 2: p7_02-p7_03/p7_07, діти з особливими освітніми потребами
## Група 3: p7_06, бездомні

In [4]:
dict_weights = {
    'p7_01':1,
    'p7_02':1,
    'p7_03':1,
    'p7_06':1,
    'p7_07':1
}

sources = [df_P07_002, df_P07_004_fin]
save_data(sources, dict_weights, "P7", show_results=True)

,region,p7_01,p7_02,p7_06,P7
0,Вінницька,0.328904,0.487799,0.004341,1.642088
1,Волинська,0.067247,0.466196,0.004744,1.076374
2,Дніпропетровська,0.120130,NaN,NaN,NaN
3,Донецька,0.122811,NaN,NaN,NaN
4,Житомирська,0.008852,0.500389,0.003964,1.026410
5,Закарпатська,0.012029,NaN,NaN,NaN
6,Запорізька,0.435616,NaN,NaN,NaN
7,Івано-Франківська,0.067085,0.377013,0.005134,0.898463
8,Київська,1.000000,NaN,NaN,NaN
9,Кіровоградська,0.095861,NaN,NaN,NaN
